In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [3]:
PATH = "data/house-prices-advanced-regression-techniques/"

In [4]:
df_raw = pd.read_feather('tmp/house_prices_raw')

In [5]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')

In [9]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 400
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

In [10]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((1060, 83), (1060,), (400, 83), (400,))

In [11]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [13]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[14327.774636025835, 33695.65131958239, 0.96822520728842842, 0.8069858216956991]


In [14]:
m = RandomForestRegressor(n_estimators=30, min_samples_leaf=2, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[14354.878210464949, 32483.317290174447, 0.96810487806089141, 0.82062486133187784, 0.84128237655147708]


In [23]:
m = RandomForestRegressor(n_estimators=45, min_samples_leaf=5, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[20729.89676032233, 32404.473637697964, 0.93348501141960605, 0.82149456484705907, 0.85102226680165172]


In [25]:
m = RandomForestRegressor(n_estimators=45, min_samples_leaf=5, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(df_trn, y_trn)
m.score(df_trn,y_trn)

0.93320377271178967

In [26]:
p = pd.read_csv(f'{PATH}test.csv', low_memory=False)
apply_cats(p, df_raw)
pred, _, nas = proc_df(p)

In [27]:
df_trn.shape, pred.shape

((1460, 83), (1459, 91))

In [28]:
pred.drop(['BsmtFinSF1_na','BsmtFinSF2_na','BsmtUnfSF_na','TotalBsmtSF_na','BsmtFullBath_na','BsmtHalfBath_na','GarageCars_na','GarageArea_na'], axis=1, inplace=True)

In [30]:
df_trn.shape, pred.shape

((1460, 83), (1459, 83))

In [31]:
new_preds = m.predict(pred)

In [32]:
pd.DataFrame({'Id':pred.Id, 'SalePrice':new_preds}).set_index('Id').to_csv('predictions/house_prices/sample_submission.csv')